# GENERATE A BACKEND PROJECT 

+ this notebook is a template to generate Backend-required files to view a project with the DataDiVR (preview or VR)
+ STEP 1 and the "create a graph" section contains a template graph writing a required format (json) to then use the generate-project functions of the DataDiVR backend

+ STEP 2 to actually generate BACKEND project files.

### THE GRAPH STRUCTURE FORMAT required

**How this is meant to be used:**
+ Create an nx.Graph with stored node positions ("pos") and colors ("nodecolor") and link colors ("linkcolor")
+ set attributes in the nx.Graph accordingly
+ then merge all layouts (nx.Graphs) using the "merge_grpahs" function to create one large file, containing all information for your project. 

In [74]:
'''

{
----------------------------------------
THIS IS THE GENERAL GRAPH INFO SECTION
----------------------------------------
  "directed": false,
  "multigraph": false,
  "graphtitle": "Testgraph",
  "graphdesc": "A toy graph for testing purposes. Number of nodes: 10, Links: 43.",
  "graphlayouts": [
      "layout1-spring",
      "layout2-spring",
      "layout3-spring",
      "layout4-clusters"
  ],
  "annotationTypes": true,
  "nodes": [
   ----------------------------------------
   contains all nodes of the project
   ----------------------------------------
      {
          "id": 0,
          "name": 0,
          "annotation": 
                {
                    "annot1": [
                        "lambda",
                        "alpha",
                        "zeta",
                        "theta"
                    ],
                    "annot2": [
                        "delta",
                        "nu"
                    ],
                    "annot3": [
                        "mu",
                        "gamma"
                    ]
          }
      },....
  ],
  "links": [
   ----------------------------------------
   contains all links of the project
   ----------------------------------------
      {
          "id": 0,
          "source": 0,
          "target": 1
      },
      {
          "id": 1,
          "source": 0,
          "target": 2
      },...
       ],
  "layouts": [
   ----------------------------------------
   contains all layouts of the project
   only contains nodes and links as well as colors specific to the layout
   ----------------------------------------
       {
          "nodes": [
              {
                  "nodecolor": [
                      255,
                      35,
                      0,
                      120
                  ],
                  "pos": [
                      -0.5618057865250979,
                      0.1467411221839164,
                      0.49656801102094605
                  ],
                  "cluster": "",
                  "id": 0
               },...
        	],
          "links": [
              {
                  "linkcolor": [
                      0,
                      255,
                      0,
                      100
                  ],
                  "source": 0,
                  "target": 1
              },...
         	],
   	  }, {
          "nodes": [
              {
                  "nodecolor": "#0000ffaa",
                  "pos": [
                      -0.35948900932978317,
                      0.6255258442839948,
                      -0.04209289102217994
                  ],
                  "cluster": "cluster group 1",
                  "id": 0
               },... 
],
          "links": [
              {
                  "linkcolor": "#0000ff",
                  "source": 0,
                  "target": 1
              },
],
  	   }, { . . .  
 	},
}

'''

'\n\n{\n----------------------------------------\nTHIS IS THE GENERAL GRAPH INFO SECTION\n----------------------------------------\n  "directed": false,\n  "multigraph": false,\n  "graphtitle": "Testgraph",\n  "graphdesc": "A toy graph for testing purposes. Number of nodes: 10, Links: 43.",\n  "graphlayouts": [\n      "layout1-spring",\n      "layout2-spring",\n      "layout3-spring",\n      "layout4-clusters"\n  ],\n  "annotationTypes": true,\n  "nodes": [\n   ----------------------------------------\n   contains all nodes of the project\n   ----------------------------------------\n      {\n          "id": 0,\n          "name": 0,\n          "annotation": [\n                {\n                    "annot1": [\n                        "lambda",\n                        "alpha",\n                        "zeta",\n                        "theta"\n                    ],\n                    "annot2": [\n                        "delta",\n                        "nu"\n                    ],\

# STEP 1 - Create Graphs with stored positions, colors etc. or use your own nx.Graphs

In [88]:
def merge_graphs(graphs):
    all_nodes = []
    all_links = []
    layouts = []

    seen_nodes = set()  # To track seen node IDs
    seen_links = set()  # To track seen links by a tuple of (source, target)

    for graph in graphs:
        # Process nodes for global and layout-specific lists
        for node, attrs in graph.nodes(data=True):
            if node not in seen_nodes:
                annotation = attrs.get('annotation', [])
                anntation_mod = {}
                for k,v in annotation.items():
                    anntation_mod[k] = v
                all_nodes.append({'id': node, 'name': node, 
                                    'annotation': anntation_mod})
                seen_nodes.add(node)

        # Process links for global list, now with separate source and target
        for ix,(source, target, attrs) in enumerate(graph.edges(data=True)):
            if (source, target) not in seen_links:
                # Assuming 'weight' might be an attribute you're interested in
                # weight = attrs.get('weight', 1)  # Example, uncomment if needed
                all_links.append({
                    'id': ix,  # It's unclear how 'id' is determined; might need a unique strategy
                    # 'w': weight,  # Uncomment if using weight
                    'source': source,
                    'target': target
                })
                seen_links.add((source, target))

        # Prepare layout-specific nodes and links
        layout_nodes = [{'nodecolor': attrs.get('nodecolor', ''),
                         'pos': attrs.get('pos', []),
                         'cluster': attrs.get('cluster',''),
                         'id': node} for node, attrs in graph.nodes(data=True)
                        ]
        layout_links = [{'linkcolor': attrs.get('linkcolor', ''),
                         'source': source,
                         'target': target} for source, target, attrs in graph.edges(data=True)]

        layouts.append({'nodes': layout_nodes, 'links': layout_links})

    # Assuming the structure of the graphs are similar, and using the first graph as the base
    graphlayouts = [graph.name for graph in graphs]
    merged_structure = {
        'directed': graphs[0].is_directed(),
        'multigraph': graphs[0].is_multigraph(),
        'graphtitle': graphs[0].graph.get("graphtitle", ""),
        'graphdesc': graphs[0].graph.get("graphdesc", ""),
        'graphlayouts': graphlayouts,
        'annotationTypes': True,
        'nodes': all_nodes,
        'links': all_links,
        'layouts': layouts
    }

    return merged_structure 

### nx.Graph

In [96]:
import networkx as nx
import json 
import os

G = nx.random_geometric_graph(10,1)
print("Number of nodes: ", len(G.nodes()))
print("Number of Links: ", len(G.edges()))

# ===============================================
# GRAPH NAME AND DESCRIPTION - a string each
# ===============================================
G.name = "A-testgraph2"
G.graph['graphtitle'] = G.name
G.graph['graphdesc'] = "A toy graph for testing purposes. Number of nodes: "+str(len(G.nodes()))+", Links: "+ str(len(G.edges()))+"."

Number of nodes:  10
Number of Links:  40


### create node anntotations

In [97]:
import random 
def generate_random_words(num):
    words = ["alpha", "beta", "gamma", "delta", "epsilon", "zeta", "theta", "lambda", "mu", "nu"]
    return random.sample(words, num)

# Create a list to hold annotations in JSON format
l_annotations_json = []

# Process each node in the graph
for g in G.nodes():
    # Generate random annotations
    annotations = {
        "annot1": generate_random_words(random.randint(3, 4)),
        "annot2": generate_random_words(random.randint(1, 2)),
        "annot3": generate_random_words(random.randint(2, 3))
    }
    
    l_annotations_json.append(annotations)

# Create a dictionary mapping nodes to their annotations
d_annotations = dict(zip(G.nodes(), l_annotations_json))

# Set the node attributes in the graph
nx.set_node_attributes(G, d_annotations, name="annotation")


### create node positions and set as "pos" Graph attribute 
here are 3 different layouts, which all are stored in unique nx.Graph-objects (G_rgba, G_hex, ....)

In [98]:
# First layout (i.e. Graph 1)
G_rgba = G.copy()
G_rgba.name = 'layout1-spring'
G_rgba.graph["scene"] = G_rgba.name
posG3D_1_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=100)
posG3D_1 = {key: value.tolist() for key, value in posG3D_1_pre.items()}
nx.set_node_attributes(G_rgba, posG3D_1, name="pos")


# Second layout (i.e. Graph 2)
G_hex = G.copy()
G_hex.name = 'layout2-spring'
G_hex.graph["scene"] = G_hex.name
posG3D_2_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=200)
posG3D_2= {key: value.tolist() for key, value in posG3D_2_pre.items()}
nx.set_node_attributes(G_hex, posG3D_2, name="pos")


# Third layout (i.e. Graph 3)
G_hex8 = G.copy()
G_hex8.name = 'layout3-spring'
G_hex8.graph["scene"] = G_hex8.name
posG3D_3_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=500)
posG3D_3 = {key: value.tolist() for key, value in posG3D_3_pre.items()}
nx.set_node_attributes(G_hex8, posG3D_3, name="pos")


# Fourth layout (i.e. Graph 4) - with clusters
G_clusters = G.copy()
G_clusters.name = 'layout4-clusters'
G_clusters.graph["scene"] = G_clusters.name
clustername_1 = 'cluster group 1'
clustername_2 = 'cluster group 2'
clustername_3 = 'cluster group 3'

# nodes into groups
for g in G_clusters.nodes():
    if g < len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_1
    elif g < 2 * len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_2
    else:
        G_clusters.nodes[g]['cluster'] = clustername_3

nx.set_node_attributes(G_clusters, posG3D_2, name="pos")

In [99]:
#get node attributes 
#node_attributes = nx.get_node_attributes(G_clusters, 'pos')
#print(node_attributes)

#### node and link colors 

In [100]:
# 3 Formats of colors values are supported: hex, rgba, hex8

# FIRST GRAPH - rgba color values
d_nodecolors_rgba = dict(zip(G_rgba.nodes(),[(255,35,0,120)]*len(G_rgba.nodes())))
nx.set_node_attributes(G_rgba, d_nodecolors_rgba, name="nodecolor")
l_linkcolors_rgba = (0,255,0,100)
nx.set_edge_attributes(G_rgba, l_linkcolors_rgba, name="linkcolor")


# SECOND GRAPH - hex color values 
d_nodecolors_hex = dict(zip(G_hex.nodes(),['#FF2300']*len(G_hex.nodes())))
nx.set_node_attributes(G_hex, d_nodecolors_hex, name="nodecolor")
l_linkcolors_hex = '#ff0000'
nx.set_edge_attributes(G_hex, l_linkcolors_hex, name="linkcolor")


# THIRD GRAPH - hex8 color values
d_nodecolors_hex8 = dict(zip(G_hex8.nodes(),['#0000ffaa']*len(G_hex8.nodes())))
nx.set_node_attributes(G_hex8, d_nodecolors_hex8, name="nodecolor")
l_linkcolors_hex8 = '#0080ffaa'
nx.set_edge_attributes(G_hex8, l_linkcolors_hex8, name="linkcolor")


# FOURTH GRAPH - clusters assigned 

# node colors 
d_nodecolors_clusters = {}
nodes_group1 = []
nodes_group2 = []
nodes_group3 = []
for n in G_clusters.nodes(): 
    if G_clusters.nodes[n]['cluster'] == clustername_1:
        d_nodecolors_clusters[n] = '#0000ffaa'
        nodes_group1.append(n)
    elif G_clusters.nodes[n]['cluster'] == clustername_2:
        d_nodecolors_clusters[n] = '#00ff00aa'
        nodes_group2.append(n)
    elif G_clusters.nodes[n]['cluster'] == clustername_3:
        d_nodecolors_clusters[n] = '#ff0000aa'
        nodes_group3.append(n)

# link colors
d_linkcolors_clusters = {}
for edge in G_clusters.edges():
    if edge[0] in nodes_group1 and edge[1] in nodes_group1:
        d_linkcolors_clusters[edge] = '#0000ff'
       
    elif edge[0] in nodes_group2 and edge[1] in nodes_group2:
        d_linkcolors_clusters[edge] = '#00ff00'
       
    elif edge[0] in nodes_group3 and edge[1] in nodes_group3:
        d_linkcolors_clusters[edge] = '#ff0000'
       
    else:
        d_linkcolors_clusters[edge] = '#B1B1B1'

l_linkcolors_clusters = list(d_linkcolors_clusters.values())

nx.set_node_attributes(G_clusters, d_nodecolors_clusters, name="nodecolor")
nx.set_edge_attributes(G_clusters, {edge: color for edge, color in zip(G_clusters.edges(), l_linkcolors_clusters)}, "linkcolor")


### merge Graph files into one

In [101]:
Graphs = [G_rgba, G_hex, G_hex8, G_clusters]
G_merged = merge_graphs(Graphs)

# store merged json 
with open('_jsontemp/merged_'+G_merged["graphtitle"]+'.json', 'w') as f:
    json.dump(G_merged, f, indent=4)

# STEP 2 - CREATE PROJECT - for DataDiVR BACKEND

In [102]:
from uploaderGraph import upload_filesJSON

upload_filesJSON(G_merged)

C_DEBUG: is dict - Uploading via Notebook function
C_DEBUG: namespace:  A-testgraph2
Successfully created the directory static/projects/A-testgraph2 
['JSON_barbellgraph', 'Human Interactome', 'JSON_autocore', 'JSON_Zachary', 'A-testgraph2', 'A-testgraph']


'<a style="color:green;">SUCCESS </a>LayoutID0 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID2 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID3 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID0 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID2 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID3 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID0 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID2 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID3 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID0_links Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1_links Link Textures Crea